In [7]:
def generateSaberToFrame(frame):
    B, G, R = {'min': 0, 'max': 255}, {'min': 0, 'max': 255}, {'min': 0, 'max': 10}
    
    #definir os intervalos de cores que vão aparecer na imagem final
    lowerColor = np.array([B['min'], G['min'], R['min']])
    upperColor = np.array([B['max'], G['max'], R['max']])
    
    #marcador pra saber se o pixel pertence ao intervalo ou não
    mask = cv2.inRange(frame, lowerColor, upperColor)
    
    #aplica máscara que "deixa passar" pixels pertencentes ao intervalo, como filtro
    result = cv2.bitwise_and(frame, frame, mask = mask)
    
    #aplica limiarização
    gray = cv2.cvtColor(result, cv2.COLOR_BGR2GRAY)
    _,gray = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    
    #preenche gaps
    kernel = np.ones((28, 28), 'uint8')
    gray = cv2.dilate(gray, kernel, iterations=3)
    gray = cv2.erode(gray, kernel, iterations=1)
   
    #adiciona todos os canais de cor à imagem limiarizada
    saber = cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)

    #canais de cores
    zeros = np.zeros(frame.shape[:2], dtype = 'uint8')
    red = zeros
    green = zeros+185
    blue = zeros+185

    #cria blur vermelho
    blur = saber
    blur = cv2.bitwise_not(blur)
    blur[:,:,0] = cv2.add(blur[:,:,0],blue)
    blur[:,:,1] = cv2.add(blur[:,:,1],green)
    blur[:,:,2] = cv2.add(blur[:,:,2],red)
    blur = cv2.bitwise_not(blur)
    blur = cv2.GaussianBlur(blur, (49, 49), 0)
    
    #diminui parte branca do sabre
    kernel = np.ones((10, 10), 'uint8')
    saber = cv2.erode(saber, kernel, iterations=3)
    
    #add sabre e blur ao frame
    sa = cv2.add(saber, frame)
    sa = cv2.add(blur, sa)
    
    return sa

In [8]:
import numpy as np
import cv2

cap = cv2.VideoCapture('vassoura.mp4')

In [9]:
w = int(cap.get(3))
h = int(cap.get(4))

out = cv2.VideoWriter('lightsaber.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 30, (w,h))

success, frame = cap.read()
while success:
    frame = generateSaberToFrame(frame)
    out.write(frame)
    frame = cv2.resize(frame, (600, 360))
    cv2.imshow('saber', frame)
    
    success, frame = cap.read()
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break      
cap.release()
out.release()
cv2.destroyAllWindows()

In [10]:
import moviepy.editor as mp
   
clip = mp.VideoFileClip('lightsaber.avi')
clip = clip.subclip(0, 22)
audioclip = mp.AudioFileClip('effect.mp3').subclip(0, 22)
videoclip = clip.set_audio(audioclip)
videoclip.ipython_display()